In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

# Install Flash Attention 2 for softcapping support
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [

    "unsloth/gemma-2-9b-bnb-4bit",

]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-9b",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Your Flash Attention 2 installation seems to be broken?
A possible explanation is you have a new CUDA version which isn't
yet compatible with FA2? Please file a ticket to Unsloth or FA2.
We shall now use Xformers instead, which does not have any performance hits!
We found this negligible impact by benchmarking on 1x A100.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: If you want to finetune Gemma 2, install flash-attn to make it faster!
To install flash-attn, do the below:

pip install --no-deps --upgrade "flash-attn>=2.6.3"
==((====))==  Unsloth 2025.1.1: Fast Gemma2 patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.9. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslotha

model.safetensors:   0%|          | 0.00/6.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 999,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.1.1 patched 42 layers with 42 QKV layers, 42 O layers and 42 MLP layers.


In [4]:
from datasets import load_dataset

# Define the prompt template
hindi_alpaca_prompt = """नीचे एक निर्देश दिया गया है जो एक कार्य का वर्णन करता है, और इसके साथ एक इनपुट है जो अतिरिक्त संदर्भ प्रदान करता है। एक उत्तर लिखें जो अनुरोध को सही ढंग से पूरा करता हो।

### निर्देश:
{}

### इनपुट:
{}

### उत्तर:
{}"""

EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

# Function to format prompts
def formatting_prompts_func(examples):
    conversations = examples["conversations"]
    texts = []
    for convo in conversations:
        instruction = convo[0]["value"]  # From "human"
        response = convo[1]["value"]     # From "gpt"
        # Since some conversations may not have explicit input, handle it gracefully
        input_text = ""  # Assume empty input unless the dataset provides otherwise
        text = hindi_alpaca_prompt.format(instruction, input_text, response) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

# Load the FreedomIntelligence/alpaca-gpt4-hindi dataset
dataset = load_dataset("FreedomIntelligence/alpaca-gpt4-hindi", split="train")
dataset = dataset.map(formatting_prompts_func, batched=True)

# Now you can use the formatted dataset for training
print(dataset["text"][:5])  # Check a few formatted examples


README.md:   0%|          | 0.00/124 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


alpaca-gpt4-hindi.json:   0%|          | 0.00/102M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/49969 [00:00<?, ? examples/s]

Map:   0%|          | 0/49969 [00:00<?, ? examples/s]

['नीचे एक निर्देश दिया गया है जो एक कार्य का वर्णन करता है, और इसके साथ एक इनपुट है जो अतिरिक्त संदर्भ प्रदान करता है। एक उत्तर लिखें जो अनुरोध को सही ढंग से पूरा करता हो।\n\n### निर्देश:\nकुछ एक रीसाइक्लिंग अभियान के लिए एक नारा सुझाव दें।\n\n\n### इनपुट:\n\n\n### उत्तर:\n1. "ग्रीन भविष्य के लिए एक साथ: कम करें, पुन: उपयोग करें, रीसाइकल करें।"\n2. "एक बेहतर कल के लिए आज ही रीसाइकल करें।"\n3. "अपने कचरे को खजाना बनाएं - रीसाइकल करें!"\n4. "जीवन के चक्र के लिए रीसाइकल करें।"\n5. "संसाधन बचाएं, अधिक रीसाइकल करें।"<eos>', 'नीचे एक निर्देश दिया गया है जो एक कार्य का वर्णन करता है, और इसके साथ एक इनपुट है जो अतिरिक्त संदर्भ प्रदान करता है। एक उत्तर लिखें जो अनुरोध को सही ढंग से पूरा करता हो।\n\n### निर्देश:\nअपने देश के राष्ट्रपति का विशिष्ट और हास्यास्पद वर्णन बनाएं।\n\n\n### इनपुट:\n\n\n### उत्तर:\nमैं एक AI हूँ, इसलिए मैं किसी भी देश से संबंधित नहीं हूँ और इसलिए कोई विशिष्ट राष्ट्रपति वर्णन नहीं कर सकता। हालांकि, अधिक जानकारी प्रदान करने पर मैं आपको अपने देश के राष्ट्रपति का एक अलग और हा

In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/49969 [00:00<?, ? examples/s]

In [6]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 49,969 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 54,018,048


Step,Training Loss
1,1.606200
2,1.783400
3,1.955100
4,1.698400
5,1.411300
6,1.570600
7,1.384700
8,1.344500
9,1.109600
10,1.312500


In [7]:
# Define the Hindi alpaca prompt (same as used during training)
hindi_alpaca_prompt = """नीचे एक निर्देश दिया गया है जो एक कार्य का वर्णन करता है, और इसके साथ एक इनपुट है जो अतिरिक्त संदर्भ प्रदान करता है। एक उत्तर लिखें जो अनुरोध को सही ढंग से पूरा करता हो।

### निर्देश:
{}

### इनपुट:
{}

### उत्तर:
{}"""

# Enable faster inference
FastLanguageModel.for_inference(model)

# List of questions in Hindi (history, science, and general knowledge)
questions = [
    hindi_alpaca_prompt.format("भारत के स्वतंत्रता संग्राम में 1857 का विद्रोह क्यों महत्वपूर्ण था?", "", ""),
    hindi_alpaca_prompt.format("पृथ्वी के वायुमंडल में कौन-कौन से प्रमुख गैसें पाई जाती हैं?", "", ""),
    hindi_alpaca_prompt.format("भारत का सबसे बड़ा राज्य कौन सा है?", "", ""),
    hindi_alpaca_prompt.format("डायनासॉर पृथ्वी से कब और कैसे विलुप्त हुए?", "", ""),
    hindi_alpaca_prompt.format("सौर मंडल में सबसे बड़ा ग्रह कौन सा है?", "", "")
]

# Tokenize the input questions
inputs = tokenizer(questions, return_tensors="pt", padding=True, truncation=True).to("cuda")

# Generate the outputs for all questions
outputs = model.generate(**inputs, max_new_tokens=256, use_cache=True)

# Decode and print the generated responses
responses = tokenizer.batch_decode(outputs, skip_special_tokens=True)
for i, response in enumerate(responses):
    print(f"प्रश्न {i+1}: {questions[i].split('### निर्देश:')[1].split('### इनपुट:')[0].strip()}")
    print(f"उत्तर: {response.split('### उत्तर:')[1].strip()}")
    print("-" * 50)


/usr/local/lib/python3.10/dist-packages/unsloth/kernels/utils.py:369: UserWarning: An output with one or more elements was resized since it had shape [1, 5, 3584], which does not match the required output shape [5, 1, 3584]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:28.)
  out = torch.matmul(X, W, out = out)


प्रश्न 1: भारत के स्वतंत्रता संग्राम में 1857 का विद्रोह क्यों महत्वपूर्ण था?
उत्तर: 1857 का विद्रोह भारत के स्वतंत्रता संग्राम में महत्वपूर्ण था क्योंकि यह ब्रिटिश शासन के खिलाफ पहला बड़े पैमाने पर विद्रोह था। यह विद्रोह भारतीयों के बीच ब्रिटिश शासन के खिलाफ असंतोष को उजागर करता था और ब्रिटिश शासन के खिलाफ एक राष्ट्रीय आंदोलन को प्रेरित करता था। विद्रोह ने ब्रिटिश शासन के खिलाफ एक राष्ट्रीय आंदोलन को प्रेरित किया, जो अंततः भारत की स्वतंत्रता के लिए नेतृत्व किया।
--------------------------------------------------
प्रश्न 2: पृथ्वी के वायुमंडल में कौन-कौन से प्रमुख गैसें पाई जाती हैं?
उत्तर: पृथ्वी के वायुमंडल में मुख्य रूप से नाइट्रोजन (78%), ऑक्सीजन (21%), आर्गन (0.93%), कार्बन डाइऑक्साइड (0.04%), और अन्य ट्रैक गैसें जैसे कि क्रिप्टन, ज़ीनॉन, और मेथेन शामिल हैं।
--------------------------------------------------
प्रश्न 3: भारत का सबसे बड़ा राज्य कौन सा है?
उत्तर: भारत का सबसे बड़ा राज्य राजस्थान है।
--------------------------------------------------
प्रश्न 4: डायनासॉर पृथ्वी से कब और क

In [9]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define the folder path in Google Drive
drive_path = '/content/drive/My Drive/Gemma_LoRA_Model/lora_model'

# Create the folder if it doesn't exist
os.makedirs(drive_path, exist_ok=True)

# Save the model and tokenizer
model.save_pretrained(drive_path)
tokenizer.save_pretrained(drive_path)

# Verify
print("Model and tokenizer saved to:", drive_path)


Mounted at /content/drive
Model and tokenizer saved to: /content/drive/My Drive/Gemma_LoRA_Model/lora_model
